In [1]:
"""make variations of input image"""
import os
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
sys.path.append(os.path.abspath("/raid/home/himanshus/experiments/repos/stable-diffusion/blip"))

import argparse, os, sys, glob
import PIL
import torch

# original_backward = torch.Tensor.backward

# def patched_backward(tensor, *args, **kwargs):
#     kwargs['allow_unused'] = True
#     original_backward(tensor, *args, **kwargs)

# torch.Tensor.backward = patched_backward
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm, trange
from itertools import islice
from einops import rearrange, repeat
from torchvision.utils import make_grid, save_image
from torch import autocast
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms.functional import InterpolationMode
from torch.utils.data import DataLoader,Dataset
from contextlib import nullcontext
import time
from pytorch_lightning import seed_everything
import robustbench
from robustbench.data import load_cifar10, load_imagenet
from robustbench.utils import load_model, clean_accuracy

from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler
from scripts.custom_Dataset import CustomDataset
from IPython.display import display, clear_output
from custom_pgd import PGD
from blip.models.blip import blip_decoder

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"
# from PIL import Image
# import requests
# import torch
# import torchvision
# from torchvision import transforms
# from torch.utils.data import Dataset, DataLoader
# from torchvision.transforms.functional import InterpolationMode
# import matplotlib.pyplot as plt
# import numpy as np
# import foolbox as fb
# from robustbench.data import load_cifar10

In [2]:
class parser():
    def __init__(self):
        self.prompt = "a painting of a virus monster playing guitar"
        self.init_img = "outputs/img2img-samples/grid-0003.png" #path to the input image
        self.outdir = "../BLIP/experiment_1/imagenet/clean_test_generated_2048" #dir to write results to
        self.skip_grid = True
        self.skip_save = False
        self.ddim_steps = 50 #number of ddim sampling steps
        self.plms = False
        self.fixed_code = False
        self.ddim_eta = 0.0 #ddim eta (eta=0.0 corresponds to deterministic sampling
        self.n_iter = 1
        self.C = 4 #latent channels
        self.f = 8
        self.n_samples = 32 #batch size
        self.n_rows = 0
        self.scale = 5.0 #unconditional guidance scale: eps = eps(x, empty) + scale * (eps(x, cond) - eps(x, empty))
        self.strength = 0.5 #strength for noising/unnoising. 1.0 corresponds to full destruction of information in init image
        self.from_file = ""
        self.config = "configs/stable-diffusion/v1-inference.yaml"
        self.ckpt = "models/ldm/stable-diffusion-v1/model.ckpt"
        self.seed = 42
        self.precision = "autocast" #["full", "autocast"]
        

In [3]:
def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())


def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)
    model.cuda()
    model.eval()
    return model


def load_img(path):
    image = Image.open(path).convert("RGB")
    w, h = image.size
    print(f"loaded input image of size ({w}, {h}) from {path}")
    w, h = map(lambda x: x - x % 64, (w, h))  # resize to integer multiple of 32
    image = image.resize((w, h), resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

In [4]:
from blip.models.blip import blip_decoder

image_size = 384
# image = load_demo_image(image_size=image_size, device=device)

model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth'
model_path = "./blip/checkpoints/model_base_capfilt_large.pth"
    
blip_model = blip_decoder(pretrained=model_path, image_size=image_size, vit='base')
blip_model.eval()
blip_model = blip_model.to(device)

reshape position embedding from 196 to 576
load checkpoint from ./blip/checkpoints/model_base_capfilt_large.pth


In [5]:
opt = parser()
seed_everything(opt.seed)

config = OmegaConf.load(f"{opt.config}")
model = load_model_from_config(config, f"{opt.ckpt}")

model = model.to(device)

Global seed set to 42


Loading model from models/ldm/stable-diffusion-v1/model.ckpt
Global Step: 840000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.2.self_attn.k_proj.weight', 'vision_model.encoder.layers.10.layer_norm1.weight', 'vision_model.encoder.layers.2.layer_norm2.bias', 'vision_model.encoder.layers.21.self_attn.q_proj.weight', 'vision_model.encoder.layers.19.self_attn.k_proj.bias', 'vision_model.encoder.layers.7.self_attn.q_proj.weight', 'vision_model.encoder.layers.20.self_attn.out_proj.bias', 'vision_model.encoder.layers.5.self_attn.k_proj.bias', 'vision_model.encoder.layers.2.self_attn.out_proj.weight', 'vision_model.encoder.layers.17.mlp.fc1.weight', 'vision_model.encoder.layers.17.self_attn.out_proj.weight', 'vision_model.encoder.layers.19.mlp.fc2.weight', 'vision_model.encoder.layers.16.self_attn.k_proj.bias', 'vision_model.encoder.layers.16.layer_norm2.bias', 'vision_model.encoder.layers.13.layer_norm2.weight', 'vision_model.encoder.layers.4.mlp.fc1.weight', 'vision_model

In [6]:
#added by himanshu for adding batch processing
blip_normalize = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
classifier_normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
classifier_transform = transforms.Compose([transforms.ToTensor()])
                                # transforms.Resize((96,96)),])
                                # classifier_normalize])
transform_blip = transforms.Compose([transforms.Resize(size=(384,384)),
                                blip_normalize
                                ])
classifier_inv_normalize = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                                     std = [ 1., 1., 1. ]),
                               ])
blip_inv_normalize = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1./0.26862954, 1./0.26130258, 1./0.27577711 ]),
                                transforms.Normalize(mean = [ -0.48145466, -0.4578275, -0.40821073 ],
                                                     std = [ 1., 1., 1. ]),])
                                # transforms.Resize((32,32))
                            #    ])
inverse_resize = transforms.Compose([transforms.Resize((32,32))])
# inverse_resize_2 = transforms.Compose([transforms.Resize((32,32))])
batch_size = opt.n_samples
num_classes = 100
# image_folder = '../BLIP/experiment_1/cifar10/clean_test/'
# label_file = '../BLIP/experiment_1/cifar10/clean_test_labels.txt'
image_folder = '../BLIP/experiment_1/imagenet/clean_test_2048'
label_file = '../BLIP/experiment_1/imagenet/clean_test_labels_2048.txt'
# captions_file = "../BLIP/experiment_1/cifar10/clean_test_captions.txt"
# test_dataset = CustomDataset(image_folder, label_file, transform=classifier_transform)
test_dataset = datasets.CIFAR100("../data/", train=False, transform=classifier_transform, download=True)
# train_dataset = datasets.STL10(root="../data",
#                            split="train",
#                            transform=transform,
#                           download=True)
# test_dataset = datasets.STL10(root="../data",
#                          split="test",
#                          transform=transform)

# train_loader = torch.utils.data.DataLoader(train_dataset,
#                                        batch_size=batch_size,
#                                        shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                     shuffle=False, num_workers=8)

# class_names = train_dataset.classes
# data_iterator = iter(train_loader)
# batch = next(data_iterator)

Files already downloaded and verified


In [7]:
if opt.plms:
    raise NotImplementedError("PLMS sampler not (yet) supported")
    sampler = PLMSSampler(model)
else:
    sampler = DDIMSampler(model)

os.makedirs(opt.outdir, exist_ok=True)
outpath = opt.outdir

batch_size = opt.n_samples
n_rows = opt.n_rows if opt.n_rows > 0 else batch_size

In [8]:
# data = [["This is a {}".format(class_names[i]) for i in batch[1]]]
# if not opt.from_file:
#     prompt = opt.prompt
#     assert prompt is not None
#     data = [batch_size * [prompt]]
# print(data, type(data))
# exit()

# else:
#     print(f"reading prompts from {opt.from_file}")
#     with open(opt.from_file, "r") as f:
#         data = f.read().splitlines()
#         data = list(chunk(data, batch_size))

# sample_path = os.path.join(outpath, "samples")
# os.makedirs(sample_path, exist_ok=True)
# base_count = len(os.listdir(sample_path))
# grid_count = len(os.listdir(outpath)) - 1

# assert os.path.isfile(opt.init_img)
# init_image = load_img(opt.init_img).to(device)
# init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)

# init_image = batch[0].to(device)
# print(init_image.shape, type(init_image))
# exit()
# init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space

# sampler.make_schedule(ddim_num_steps=opt.ddim_steps, ddim_eta=opt.ddim_eta, verbose=False)

assert 0. <= opt.strength <= 1., 'can only work with strength in [0.0, 1.0]'
t_enc = int(opt.strength * opt.ddim_steps)
print(f"target t_enc is {t_enc} steps")

precision_scope = autocast if opt.precision == "autocast" else nullcontext

target t_enc is 25 steps


In [9]:
mean = [0.48145466, 0.4578275, 0.40821073]
std = [0.26862954, 0.26130258, 0.27577711]
def inverse_normalize(batch_tensor, mean=mean, std=std):
    for tensor, m, s in zip(batch_tensor, mean, std):
        tensor.mul_(s).add_(m)
    return batch_tensor

def defense(img):
    x_samples = None
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                tic = time.time()
                # clear_output(wait=True)
                init_image = transform_blip(img).to(device)
                # data = [list(batch[2])]
                # print(data)
                data = [blip_model.generate(init_image, sample=False, num_beams=3, max_length=20, min_length=5)]
                # print(data)
                num = len(img)
                init_image = inverse_normalize(init_image)
                init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space
                sampler.make_schedule(ddim_num_steps=opt.ddim_steps, ddim_eta=opt.ddim_eta, verbose=False)
                for n in range(opt.n_iter):
                    for prompts in data:
                        uc = None
                        if opt.scale != 1.0:
                            uc = model.get_learned_conditioning(num * [""])
                        if isinstance(prompts, tuple):
                            prompts = list(prompts)
                        c = model.get_learned_conditioning(prompts)

                        # encode (scaled latent)
                        z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*num).to(device))
                        # decode it
                        samples = sampler.decode(z_enc, c, t_enc, unconditional_guidance_scale=opt.scale,
                                                unconditional_conditioning=uc,)

                        x_samples = model.decode_first_stage(samples)
                        x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)

                toc = time.time()
    return inverse_resize(x_samples)
# print(f"Your samples are ready and waiting for you here: \n{outpath} \n"
#       f" \nEnjoy.")
# if not opt.skip_save:
#     for x_sample in x_samples:
#         x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
#         Image.fromarray(x_sample.astype(np.uint8)).save(
#             os.path.join(sample_path, f"{base_count}.png"))
#         base_count += 1
# all_samples.append(x_samples)

In [11]:
# torch.save(model, "resnet50_stl10_trained_on_diffusion_images.pth")
# classifier = torch.load("wrn28_10_cifar10_trained_on_diffusion_images_50.pth")
# classifier = torch.load("resnet_50_pretrained_cifar10_trained_on_diffusion_2.pth")
# classifier = load_model(model_name='Standard_R50', dataset='imagenet', threat_model='Linf')
classifier = load_model(model_name='Standard', dataset='cifar100', threat_model='Linf')
classifier = classifier.to(device).eval()

KeyError: 'Standard'

In [ ]:
atk = PGD(classifier,eps=0.0, alpha=0.0, defense=defense)
atk.set_normalization_used(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [ ]:

count = 0
for sample in test_loader:
    # adv_images = atk(sample[0], sample[1])
    # adv_images = classifier_inv_normalize(adv_images)    
    generated_image = defense(sample[0])
    for a_i in generated_image:
        
        save_image(a_i, "../BLIP/experiment_1/imagenet/clean_test_generated_2048/{}.png".format(count))
        # ig_purified = defense(torch.unsqueeze(ig, dim=0))
        
        # save_image(ig_purified, "../BLIP/experiment_1/stl_10/adv_test_generated_e2e/cleaned/{}.png".format(count))
        count+=1

In [ ]:
ig_purified = defense(torch.unsqueeze(ig, dim=0))

In [ ]:
type(ig.cpu().numpy())

In [ ]:
npimg = np.transpose(ig.cpu().numpy(), (1,2,0))

In [ ]:
class Defense(nn.Module):
    def __init__(self,classifier, args, device, img_size=224):
        super().__init__()
        self.args = args
        self.device = device
        self.classifier = classifier.to(device)

        self.image_size = 384
        model_path = "./blip/checkpoints/model_base_capfilt_large.pth"
            
        self.blip_model = blip_decoder(pretrained=model_path, image_size=self.image_size, vit='base')
        self.blip_model.eval()
        self.blip_model = self.blip_model.to(self.device)
        seed_everything(self.args.seed)

        self.config = OmegaConf.load(f"{self.args.config}")
        self.model = self.load_model_from_config(self.config, f"{self.args.ckpt}")

        self.model = self.model.to(self.device)
        self.sampler = DDIMSampler(self.model)
        self.batch_size = self.args.n_samples
        self.t_enc = int(self.args.strength * self.args.ddim_steps)
        self.precision_scope = autocast if self.args.precision == "autocast" else nullcontext

        self.blip_normalize = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        self.transform_blip = transforms.Compose([transforms.Resize(size=(384,384)),
                                self.blip_normalize
                                ])
        self.inverse_resize = transforms.Compose([transforms.Resize((img_size,img_size))])
        mean = [0.48145466, 0.4578275, 0.40821073]
        std = [0.26862954, 0.26130258, 0.27577711]
        self.mean = torch.tensor(mean).view(1, -1, 1, 1).to(self.device)
        self.std = torch.tensor(std).view(1, -1, 1, 1).to(self.device)
        
    def load_model_from_config(self, config, ckpt, verbose=False):
        print(f"Loading model from {ckpt}")
        pl_sd = torch.load(ckpt, map_location="cpu")
        if "global_step" in pl_sd:
            print(f"Global Step: {pl_sd['global_step']}")
        sd = pl_sd["state_dict"]
        model = instantiate_from_config(config.model)
        m, u = model.load_state_dict(sd, strict=False)
        if len(m) > 0 and verbose:
            print("missing keys:")
            print(m)
        if len(u) > 0 and verbose:
            print("unexpected keys:")
            print(u)
        model.cuda()
        model.eval()
        return model
    
    def inverse_normalize(self, batch_tensor):
        unnorm_data = batch_tensor * self.std + self.mean
        return unnorm_data

    def forward(self, init_image):
        with self.precision_scope("cuda"):
            with self.model.ema_scope():
                init_image = self.transform_blip(init_image).to(self.device)
                data = [self.blip_model.generate(init_image, sample=False, num_beams=3, max_length=20, min_length=5)]
                num = len(init_image)
                init_image = self.inverse_normalize(init_image)
                init_latent = self.model.get_first_stage_encoding(self.model.encode_first_stage(init_image))  # move to latent space
                self.sampler.make_schedule(ddim_num_steps=self.args.ddim_steps, ddim_eta=self.args.ddim_eta, verbose=False)
                # for n in range(self.args.n_iter):
                for prompts in data:
                    uc = None
                    prompts = list(prompts)
                    c = self.model.get_learned_conditioning(prompts)

                    # encode (scaled latent)
                    z_enc = self.sampler.stochastic_encode(init_latent, torch.tensor([self.t_enc]*num).to(self.device))
                    # decode it
                    init_image = self.sampler.decode(z_enc, c, self.t_enc, unconditional_guidance_scale=self.args.scale,
                                            unconditional_conditioning=uc,)

                    init_image = self.model.decode_first_stage(init_image)
                    init_image = torch.clamp((init_image + 1.0) / 2.0, min=0.0, max=1.0)
        init_image = self.inverse_resize(init_image)
        init_image = self.classifier(init_image.to(torch.float16))
        return init_image

In [ ]:
args = parser()
model = load_model(model_name='Standard_R50', dataset='imagenet', threat_model='Linf').eval()
defense = Defense(model, args, device)


In [ ]:
batch_size = args.n_samples
transform = transforms.Compose([transforms.ToTensor(),])
imagenet_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])
# image_folder = '../BLIP/experiment_1/cifar10/clean_test_generated/'
# label_file = '../BLIP/experiment_1/cifar10/clean_test_labels.txt'
image_folder = '../BLIP/experiment_1/imagenet/clean_test/'
label_file = '../BLIP/experiment_1/imagenet/clean_test_labels.txt'
# image_folder = '../BLIP/experiment_1/cifar10/clean_test_generated_1_iter_50_step/'
# label_file = '../BLIP/experiment_1/cifar10/clean_test_labels.txt'
# # captions_file = "../BLIP/experiment_1/stl_10/clean_test_captions.txt"
test_dataset = CustomDataset(image_folder, label_file, transform=imagenet_transform)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=8)

In [ ]:
images_list = []
labels_list = []

for images, labels in test_loader:
    images_list.append(images)
    labels_list.append(labels)

# Concatenate all batches
x_test = torch.cat(images_list, 0)
y_test = torch.cat(labels_list, 0)

In [ ]:
classifier = classifier.to(torch.float32)
classifier(x_test[0].to(torch.float32).unsqueeze(0))
x_test[0].shape

In [ ]:
acc = clean_accuracy(defense, x_test, y_test, device=device, batch_size=16)
print(acc)

In [ ]:
from auto_attack.autoattack import AutoAttack
defense.float().eval()
adversary = AutoAttack(defense, norm="Linf", version="rand", eps=4/255, device=device, log_path="aa_log.txt")
x_adv = adversary.run_standard_evaluation(x_test[:4], y_test[:4],4)

In [ ]:
from bpdaattack import BPDAattack
def accu(output, target):
    with torch.no_grad():
        pred = torch.argmax(output, dim=1)
        target = target
        assert pred.shape[0] == len(target)
        correct = 0
        correct += torch.sum(pred == target).item()
    return correct / len(target)


total_metrics = 0
defense_metrics = 0

adversary = BPDAattack(classifier.to(torch.float16), defense, device,
                                    epsilon=4/255,
                                    learning_rate=2/255,
                                    max_iterations=40)
for batch_idx, (gt_image, label) in enumerate(tqdm(test_loader)):
    gt_image, label = gt_image.to(device).to(torch.float16), label.to(device)

    adv = adversary.generate(gt_image, label)
    adv = adv.detach()

    logits = classifier(adv.to(torch.float16))
    total_metrics += accu(logits, label)

    reformed = defense(adv)
    logits = classifier(reformed.to(torch.float16))
    defense_metrics += accu(logits, label)

total_metrics /= len(test_loader)
defense_metrics /= len(test_loader)


In [ ]:
print(total_metrics, defense_metrics)

In [ ]:
import foolbox
import torch
# from utils import *
# from purification import *

### Classifer PGD attack
# Attack input x
def bpda(x, y, network_ebm, network_clf, device="cuda", ball_dim=-1, ptb=8., alpha=2., n_eot=1, iterations=40):
    fmodel = foolbox.PyTorchModel(network_clf, bounds=(0., 1.))
    x = x.to(device).to(torch.float16)
    y = y.to(device)
    x_temp = x.clone().detach()
    for i in range(iterations):
        # get gradient of purified images for n_eot times
        grad = torch.zeros_like(x_temp).to(device)
        for j in range(n_eot):
            x_temp_eot = network_ebm(x_temp)
            if ball_dim==-1:
                attack = foolbox.attacks.LinfPGD(rel_stepsize=0.05, steps=1, random_start=False) # Can be modified for better attack
                _, x_temp_eot_d, _ = attack(fmodel, x_temp_eot, y, epsilons=ptb/255.)
            elif ball_dim==2:
                attack = foolbox.attacks.L2PGD(rel_stepsize=0.05) # Can be modified for better attack
                _, x_temp_eot_d, _ = attack(fmodel, x_temp_eot.float(), y, epsilons=ptb/255.)
            grad += (x_temp_eot_d.detach() - x_temp_eot).to(device)
        # Check attack success
        x_clf = x_temp.clone().detach().to(device)
        success = torch.eq(torch.argmax(network_clf(x_clf), dim=1), y)
        grad *= success[:, None, None, None] # Attack correctly classified images only
        x_temp = torch.clamp(x + torch.clamp(x_temp - x + grad.sign()*alpha/255., -1.0*ptb/255., ptb/255.), 0.0, 1.0)

    x_adv = x_temp.clone().detach()
    x_clf = x_adv.clone().detach().to(device)
    success = torch.eq(torch.argmax(network_clf(network_ebm(x_clf)), dim=1), y)
    acc = success.float().mean(axis=-1)

    return x_adv, success, acc


acc_list = []
for batch in tqdm(test_loader):
    x,y = batch
    _, _, acc = bpda(x,y,defense,classifier.to(torch.float16))
    print(acc)
    acc_list.append(acc)

In [ ]:
a = torch.tensor([])
b = torch.tensor([2.,4.])

In [ ]:
a = torch.cat([a, torch.tensor([acc_list[2].cpu()])])
a 

In [ ]:
torch.mean(a)

In [12]:
import foolbox
import torch

### Classifer PGD attack
# Attack input x
def clf_pgd(x, y, network_ebm, network_clf, device='cuda', ptb=8., dim=-1):
    x = x.to(device)
    y = y.to(device)
    fmodel = foolbox.PyTorchModel(network_clf, bounds=(0., 1.))
    if dim==-1:
        attack = foolbox.attacks.LinfPGD(abs_stepsize=2./255., steps=20) # Can be modified for better attack
        _, x_adv, success = attack(fmodel, x, y, epsilons=ptb/255.)
        acc = 1 - success.float().mean(axis=-1)
    elif dim==2:
        attack = foolbox.attacks.L2PGD(rel_stepsize=0.25) # Can be modified for better attack
        _, x_adv, success = attack(fmodel, x, y, epsilons=ptb/255.)
        acc = 1 - success.float().mean(axis=-1)
    return x_adv, success, acc


acc_list = torch.tensor([])
for x, y  in tqdm(test_loader):
    x_adv, _, _ = clf_pgd(x.to(torch.float16),y,defense,classifier.to(torch.float16))
    success = torch.eq(torch.argmax(classifier(defense(defense(x_adv.to(device)))), dim=1), y.to(device))
    acc = success.float().mean(axis=-1)
    acc_list = torch.concat([acc_list, torch.tensor([acc.item()])])
    print("Running:",torch.mean(acc_list), " Current batch:", acc.item())

  1%|          | 1/128 [00:06<14:34,  6.89s/it]

Running: tensor(0.5000)  Current batch: 0.5


  2%|▏         | 2/128 [00:14<15:20,  7.30s/it]

Running: tensor(0.5312)  Current batch: 0.5625


  2%|▏         | 3/128 [00:22<15:43,  7.54s/it]

...

Running: tensor(0.4238)  Current batch: 0.25


 26%|██▌       | 33/128 [03:43<10:17,  6.50s/it]

Running: tensor(0.4205)  Current batch: 0.3125


 26%|██▌       | 33/128 [03:48<10:58,  6.93s/it]


KeyboardInterrupt: 

In [12]:
images_list = []
labels_list = []
for images, labels in test_loader:
    images_list.append(images)
    labels_list.append(labels)

# Concatenate all batches
x = torch.cat(images_list, 0)
y = torch.cat(labels_list, 0)

In [17]:
from autoattack import AutoAttack
adversary = AutoAttack(classifier, norm='Linf', eps=4/255, version='standard', device=device,)
adversary.apgd.n_restarts = 1
x_adv = adversary.run_standard_evaluation(x, y, 64,)

setting parameters for standard version
using standard version including apgd-ce, apgd-t, fab-t, square.


initial accuracy: 76.76%
apgd-ce - 1/25 - 64 out of 64 successfully perturbed
apgd-ce - 2/25 - 64 out of 64 successfully perturbed
apgd-ce - 3/25 - 64 out of 64 successfully perturbed
apgd-ce - 4/25 - 64 out of 64 successfully perturbed
apgd-ce - 5/25 - 64 out of 64 successfully perturbed
apgd-ce - 6/25 - 64 out of 64 successfully perturbed
apgd-ce - 7/25 - 64 out of 64 successfully perturbed
apgd-ce - 8/25 - 64 out of 64 successfully perturbed
apgd-ce - 9/25 - 64 out of 64 successfully perturbed
apgd-ce - 10/25 - 64 out of 64 successfully perturbed
apgd-ce - 11/25 - 64 out of 64 successfully perturbed
apgd-ce - 12/25 - 64 out of 64 successfully perturbed
apgd-ce - 13/25 - 64 out of 64 successfully perturbed
apgd-ce - 14/25 - 64 out of 64 successfully perturbed
apgd-ce - 15/25 - 64 out of 64 successfully perturbed
apgd-ce - 16/25 - 64 out of 64 successfully perturbed
apgd-ce - 17/25 - 64 out of 64 successfully perturbed
apgd-ce - 18/25 - 64 out of 64 successfully perturbed
apgd-ce - 19

In [41]:
purified = torch.tensor([])
for i in x_adv:
    i = defense(i.unsqueeze(0)).to(device)
    purified = torch.concat([purified, i.detach().cpu()])

# acc = clean_accuracy(classifier, x_adv, y, 16, device=device)

In [42]:
acc = clean_accuracy(classifier, purified, y, 16, device=device)

In [44]:
acc

0.52734375

In [35]:
atk = PGD(classifier, eps=4/255, alpha=2/225, steps=10)


In [40]:
x_adv = torch.tensor([])
for i,j in zip(x,y):
    adv = atk(i.unsqueeze(0).to(device), j.unsqueeze(0).to(device))
    x_adv = torch.concat([x_adv, adv.detach().cpu()])

In [45]:
purified.shape

torch.Size([2048, 3, 224, 224])

In [13]:
from robustbench.model_zoo import cifar100

In [14]:
cifar100.cifar_100_models.

OrderedDict([(<ThreatModel.Linf: 'Linf'>,
              OrderedDict([('Gowal2020Uncovering',
                            {'model': <function robustbench.model_zoo.cifar100.<lambda>()>,
                             'gdrive_id': '16I86x2Vv_HCRKROC86G4dQKgO3Po5mT3'}),
                           ('Gowal2020Uncovering_extra',
                            {'model': <function robustbench.model_zoo.cifar100.<lambda>()>,
                             'gdrive_id': '1LQBdwO2b391mg7VKcP6I0HIOpC6O83gn'}),
                           ('Cui2020Learnable_34_20_LBGAT6',
                            {'model': <function robustbench.model_zoo.cifar100.<lambda>()>,
                             'gdrive_id': '1rN76st8q_32j6Uo8DI5XhcC2cwVhXBwK'}),
                           ('Cui2020Learnable_34_10_LBGAT0',
                            {'model': <function robustbench.model_zoo.cifar100.<lambda>()>,
                             'gdrive_id': '1RnWbGxN-A-ltsfOvulr68U6i2L8ohAJi'}),
                           ('Cui2020